In [22]:
!pip install torch soundfile


In [1]:
!pip install datasets pandas transformers accelerate huggingface_hub

In [2]:
!pip uninstall datasets -y
!pip install --upgrade datasets


Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [24]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import pipeline
from huggingface_hub import login, HfApi
import torch

In [ ]:
ds_stream = load_dataset(
    "arood0/mmm_project_with_ru_final",
    split="train"
)

processed_examples = []


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/752 [00:00<?, ?B/s]

data/train-00000-of-00012.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

data/train-00001-of-00012.parquet:   0%|          | 0.00/416M [00:00<?, ?B/s]

data/train-00002-of-00012.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

...

data/train-00008-of-00012.parquet:   0%|          | 0.00/513M [00:00<?, ?B/s]

data/train-00009-of-00012.parquet:   0%|          | 0.00/455M [00:00<?, ?B/s]

data/train-00010-of-00012.parquet:   0%|          | 0.00/439M [00:00<?, ?B/s]

data/train-00011-of-00012.parquet:   0%|          | 0.00/491M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [26]:
model, example_text = torch.hub.load(
    repo_or_dir='snakers4/silero-models',
    model='silero_tts',
    language='ru',
    speaker='v5_ru'
)


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master
100%|██████████| 139M/139M [00:06<00:00, 23.6MB/s]
/usr/local/lib/python3.12/dist-packages/torch/package/package_importer.py:257: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  dtype = storage_type.dtype


In [ ]:
import soundfile as sf
put_accent=True
put_yo=True
put_stress_homo=True
put_yo_homo=True

import torch
import soundfile as sf
import os
import random
from datasets import Audio


os.makedirs("audio_final", exist_ok=True)

voices = ["baya", "aidar", "kseniya", "eugene", "xenia"]

def tts_silero(text, path):
    voice = random.choice(voices) 
    audio = model.apply_tts(
        text=text,
        speaker=voice,
        sample_rate=48000
    )
    sf.write(path, audio, 48000)

def add_audio_column(dataset):
    audio_paths = []

    for row in dataset:
        text = row["EDITING_INSTRUCTION_RU"]
        image_id = row["IMAGE_ID"]
        path = f"audio_final/{image_id}.wav"
        tts_silero(text, path)
        audio_paths.append(path)

    dataset = dataset.add_column("audio", audio_paths)
    dataset = dataset.cast_column("audio", Audio())
    return dataset



In [43]:
tts_silero("Привет, это тест  тесто место хочу сесть хочешь!", "audio_final/test2.wav")


In [44]:
from huggingface_hub import login
login()


In [49]:
ds = add_audio_column(ds_stream)


In [50]:

ds.push_to_hub("arood0/mmm_project_with_audio_ru_final")

Uploading the dataset shards:   0%|          | 0/13 [00:00<?, ? shards/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   3%|2         | 13.3MB /  484MB            

CommitInfo(commit_url='https://huggingface.co/datasets/arood0/mmm_project_with_audio_ru_final/commit/1e456623760b8956842a3011c15d55d53cde92a2', commit_message='Upload dataset', commit_description='', oid='1e456623760b8956842a3011c15d55d53cde92a2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/arood0/mmm_project_with_audio_ru_final', endpoint='https://huggingface.co', repo_type='dataset', repo_id='arood0/mmm_project_with_audio_ru_final'), pr_revision=None, pr_num=None)